<a href="https://colab.research.google.com/github/avikumart/LLM-GenAI-Transformers-Notebooks/blob/main/TMLC_LLM_projects/RAG/RAG_Retrieval_Optimization_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain-cohere langchain pdfminer.six chromadb rank_bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.2/606.2 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.2/250.2 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 18.4 MB/s eta 

In [2]:
import os
from google.colab import userdata
os.environ["COHERE_API_KEY"] = userdata.get('COHERE_KEY')
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_cohere import ChatCohere
from langchain.schema.output_parser import StrOutputParser
from pdfminer.high_level import extract_text as extract_text_pdf_miner
from langchain.vectorstores import Chroma
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.embeddings import CohereEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_core.runnables import RunnableParallel,RunnablePassthrough

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


## Hybrid Search

In [3]:
# Define the directory where the Chroma database will persist data
persist_directory = "/content/hybrid-search"

# Initialize Cohere embeddings with the specified model
# "embed-english-v3.0" is a pre-trained English language embedding model by Cohere
# The user_agent parameter specifies the tool or library using the Cohere API, in this case, LangChain
embedding = CohereEmbeddings(
    model="embed-english-v3.0",
    user_agent="langchain"
)


<ipython-input-3-3bb00f7d6c8c>:7: LangChainDeprecationWarning: The class `CohereEmbeddings` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereEmbeddings``.
  embedding = CohereEmbeddings(


In [4]:
def load_data_to_vectordb(file_path,source):
  global bm25_retriever
  # Loop through a list of PDF files to process
  pages = []

  for pdf_name in [file_path]:
      # Open each PDF file in binary mode
      with open(pdf_name, 'rb') as f:
          # Extract text from the PDF using the extract_text_pdf_miner function
          text = extract_text_pdf_miner(f)

          # Clean the extracted text by removing newline characters and joining into a single string
          cleaned_text = " ".join(text.split("\n"))

          # Initialize a list to store document chunks
          docs = []

          # Create a text splitter to divide the text into manageable chunks
          # Each chunk has a maximum size of 2048 characters with a 512-character overlap
          splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=512)

          # Split the cleaned text into chunks and wrap each chunk in a Document object
          for chunk in splitter.split_text(cleaned_text):
              docs.append(Document(page_content=chunk, metadata={"retrived_from":source,"source": pdf_name}))
              pages.append(Document(page_content=chunk, metadata={"retrived_from":source,"source": pdf_name}))
      # Create a Chroma collection from the processed documents
      # Use the specified persist directory and embedding model for storage and retrieval
      if source == 1:
        bm25_retriever = BM25Retriever.from_documents(pages)
      else:
        db = Chroma.from_documents(
            documents=docs,
            persist_directory=persist_directory,
            embedding=embedding
        )

load_data_to_vectordb(file_path="/content/Newwhitepaper_Agents.pdf",source=1)
load_data_to_vectordb(file_path="/content/Newwhitepaper_Operationalizing Generative AI on Vertex AI.pdf",source=2)

In [5]:
# Initialize the BM25 retriever
bm25_retriever.k = 2  # Retrieve top 2 results
print("type of bm25", type(bm25_retriever))

type of bm25 <class 'langchain_community.retrievers.bm25.BM25Retriever'>


In [6]:
# Initialize retriever
docsearch = Chroma(persist_directory=persist_directory, embedding_function=embedding)
retriever_chromadb = docsearch.as_retriever(search_kwargs={"k": 5})

# Initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever_chromadb], weights=[0.3, 0.7]
)

<ipython-input-6-641eb3961709>:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  docsearch = Chroma(persist_directory=persist_directory, embedding_function=embedding)


In [7]:
# Example query
query = "What is a AI agents?"

# Retrieve relevant documents/products
docs = ensemble_retriever.invoke(query)

docs

[Document(metadata={'retrived_from': 2, 'source': '/content/Newwhitepaper_Operationalizing Generative AI on Vertex AI.pdf'}, page_content='AI Agent Builder, you are be able to easily ground your agents by pointing to a diverse range   of data sources, including structured datastores such us BigQuery, Spanner, Cloud SQL,   unstructured sources like website content crawling and cloud storage as well as connectors   to Google drive and other APIs. Agent Builder utilizes a robust foundation of Google Search   technologies, encompassing semantic search, content chunking, ranking, algorithms,   and user intent understanding. Under the hood it optimizes document loading, chunking,   embedding models, and ranking strategies. It abstracts away these complexities and allows   users to simply specify their data source to initiate the gen AI-powered agent.This approach   is ideal for organizations seeking to build robust search experiences for standard use cases   without extensive technical exper

In [8]:
#Extract and print only the page content from each document
import pandas as pd

retrieval_df = pd.DataFrame()

page_content = []
retrieval_source = []
pdf_source = []

for doc in docs:
    page_content.append(doc.page_content)
    retrieval_source.append(doc.metadata['retrived_from'])
    pdf_source.append(doc.metadata['source'])

retrieval_df['page_content'] = page_content
retrieval_df['retrieval_source'] = retrieval_source
retrieval_df['pdf_source'] = pdf_source

retrieval_df.head(10)

,page_content,retrieval_source,pdf_source
0,"AI Agent Builder, you are be able to easily gr...",2,/content/Newwhitepaper_Operationalizing Genera...
1,what arguments to use. Vertex AI Grounding5...,2,/content/Newwhitepaper_Operationalizing Genera...
2,"is required. Depending on the use case, levera...",2,/content/Newwhitepaper_Operationalizing Genera...
3,"and models used and their lineage, the code ...",2,/content/Newwhitepaper_Operationalizing Genera...
4,AI using ML OpsSeptember 2024 Chain & Augment:...,2,/content/Newwhitepaper_Operationalizing Genera...
5,a model can leverage a database retrieval to...,1,/content/Newwhitepaper_Agents.pdf
6,"Agents Authors: Julia Wiesinger, Patrick Marl...",1,/content/Newwhitepaper_Agents.pdf


In [9]:
retrieval_df.shape

(7, 3)

## Re Ranking

In [10]:
# Define the directory where the Chroma database will persist data
persist_directory = "/content/re-rank"

# Initialize Cohere embeddings with the specified model
# "embed-english-v3.0" is a pre-trained English language embedding model by Cohere
# The user_agent parameter specifies the tool or library using the Cohere API, in this case, LangChain
embedding = CohereEmbeddings(
    model="embed-english-v3.0",
    user_agent="langchain"
)

In [11]:
def load_data_to_vectordb(file_path,source):
  # Loop through a list of PDF files to process

  for pdf_name in [file_path]:
      # Open each PDF file in binary mode
      with open(pdf_name, 'rb') as f:
          # Extract text from the PDF using the extract_text_pdf_miner function
          text = extract_text_pdf_miner(f)

          # Clean the extracted text by removing newline characters and joining into a single string
          cleaned_text = " ".join(text.split("\n"))

          # Initialize a list to store document chunks
          docs = []

          # Create a text splitter to divide the text into manageable chunks
          # Each chunk has a maximum size of 2048 characters with a 512-character overlap
          splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=512)

          # Split the cleaned text into chunks and wrap each chunk in a Document object
          for chunk in splitter.split_text(cleaned_text):
              docs.append(Document(page_content=chunk, metadata={"source": pdf_name}))
      # Create a Chroma collection from the processed documents
      # Use the specified persist directory and embedding model for storage and retrieval

      db = Chroma.from_documents(
            documents=docs,
            persist_directory=persist_directory,
            embedding=embedding
        )

load_data_to_vectordb(file_path="/content/Newwhitepaper_Agents.pdf",source=1)
load_data_to_vectordb(file_path="/content/Newwhitepaper_Operationalizing Generative AI on Vertex AI.pdf",source=2)

In [12]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

In [13]:
docsearch = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [15]:
# Initialize an empty DataFrame with specified columns
non_rerank_df = pd.DataFrame(columns=['Text', 'source', 'relevance_score'])

# Perform similarity search using a preconfigured document search tool
# This retrieves the top 3 documents based on relevance to the query
res_docs = docsearch.similarity_search_with_relevance_scores("What is the AI agents?", k=3)

# Loop through the retrieved documents and populate the DataFrame
for doc in res_docs:
    non_rerank_df = non_rerank_df._append(
        {
            'Text': doc[0].page_content,  # Extract the page content (text) from the document
            'source': doc[0].metadata['source'],  # Extract the source metadata
            'relevance_score': doc[1]  # Extract the relevance score
        },
        ignore_index=True  # Ensure the new row is appended without overwriting existing rows
    )

# Display the first 3 rows of the DataFrame
non_rerank_df.head()

<ipython-input-15-79a2022a2232>:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  non_rerank_df = non_rerank_df._append(


,Text,source,relevance_score
0,"Agents Authors: Julia Wiesinger, Patrick Marl...",/content/Newwhitepaper_Agents.pdf,0.360925
1,a model can leverage a database retrieval to...,/content/Newwhitepaper_Agents.pdf,0.357930
2,of language models by leveraging tools to acce...,/content/Newwhitepaper_Agents.pdf,0.342624


In [16]:
# Import and initialize the reranker for document compression
compressor = CohereRerank()
# CohereRerank is a model or tool designed to rerank documents based on their relevance.

# Create a ContextualCompressionRetriever for improved document retrieval
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,               # Use the reranker as the base compression mechanism
    base_retriever=docsearch.as_retriever()   # Use the existing document search tool as the base retriever
)
# The ContextualCompressionRetriever combines the base retriever's results with reranking
# to provide more contextually relevant and concise results.

<ipython-input-16-f983c412f7af>:2: LangChainDeprecationWarning: The class `CohereRerank` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereRerank``.
  compressor = CohereRerank()


In [19]:
# Initialize an empty DataFrame with specified columns
source_df = pd.DataFrame(columns=['Text', 'source', 'relevance_score'])

# Retrieve compressed documents relevant to the query using the contextual compression retriever
compressed_docs = compression_retriever.get_relevant_documents("What is the AI agents and how does that work?")

# Loop through the first 3 compressed documents and populate the DataFrame
for i in range(3):
    source_df = source_df._append(
        {
            'Text': compressed_docs[i].page_content,  # Extract the content of the document
            'source': compressed_docs[i].metadata['source'],  # Extract the source information
            'relevance_score': compressed_docs[i].metadata['relevance_score']  # Extract the relevance score
        },
        ignore_index=True  # Ensure the new row is appended without overwriting existing rows
    )

# Display the first 3 rows of the DataFrame
source_df.head(3)

<ipython-input-19-911209b96348>:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  source_df = source_df._append(


,Text,source,relevance_score
0,a model can leverage a database retrieval to...,/content/Newwhitepaper_Agents.pdf,0.991938
1,"Agents Authors: Julia Wiesinger, Patrick Marl...",/content/Newwhitepaper_Agents.pdf,0.940790
2,of language models by leveraging tools to acce...,/content/Newwhitepaper_Agents.pdf,0.919787


We can see the last result is different and the relevance scores are also different but they are according to the relevance model